<a href="https://colab.research.google.com/github/rpedram/data-science/blob/main/my_library_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Rakesh Function library
These a list of my repeatedly used data science function that can either be called for directly after importing module or copied pasted in your code.
Remember to import relevant the libraries in main pgme



## Data Loading
Data loading for google drive ans summary of df. Initial eyeballing

In [ ]:
# -*- coding: utf-8 -*-
"""rak_analysis_lib.py

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1sUA0ZwTuO1xkwnyOhro_Kl-UWf0NZ04X

"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

# Test and trouble shoot module import
def test_function():
    return "Hello your module rak_analysis_lib! has been succesfully imported"
# --- EDA Functions ---
# Load my df from google drive
def load_data(data_path):
    """Loads data from a CSV or Excel file."""
    data_path='/content/drive/My Drive/Dataset/'+ data_path
    if data_path.endswith(".csv"):
        return pd.read_csv(data_path)
    elif data_path.endswith(".xlsx"):
        return pd.read_excel(data_path)
    else:
        raise ValueError("Unsupported file format. Supported formats: CSV, Excel")
# saves my df to google drive
def save_data(df,filename):
    """Saves data from a CSV or Excel file to my google drive"""
    data_path='/content/drive/My Drive/Dataset/'+ filename
    if data_path.endswith(".csv"):
        df.to_csv(data_path, index=False)
    elif data_path.endswith(".xlsx"):
        df.to_excel(data_path, index=False)
    else:
        raise ValueError("Unsupported file format. Supported formats: CSV, Excel")

def data_summary(data):
    """Provides descriptive statistics for a DataFrame."""
    print(data.describe(include="all"))
    print(data.isna().sum())  # Check for missing values
    print(data.head()) # Print the head
    print(data.tail()) # Print tail of data

"""## Descriptive deep Statistical study
Experimented with tabulating the numeric data into tables for skewness indec and normality using Wilk-Shapiro
"""

from scipy.stats import shapiro
from scipy.stats import skew
# Tail classification of data
def classify_tail(df):
    """
    Computes tail index and classifies tail behavior for each numeric column in the DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        None (Prints the table with column name, Tail Index, and Classification).
    """
    print(f"{'Column Name':<20} {'Tail Index':<12} {'Classification':<15}")
    print("-" * 45)

    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            data = df[col]
            tail_index = skew(data, bias=False)  # Using sample skewness (unbiased)

            if tail_index < -1:
                classification = "Heavy Tail"
            elif -1 <= tail_index <= 1:
                classification = "Stable"
            else:
                classification = "Light Tail"
        else:
            tail_index = "N/A"
            classification = "Not applicable"

        print(f"{col:<20} {tail_index:<12} {classification:<15}")

# Function to check normality
def check_normality(df):
    """
    Computes normality test (Shapiro-Wilk) for numeric columns in the DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame.

    Returns:
        None (Prints the table with column name, Shapiro index, and classification).
    """
    print(f"{'Column Name':<20} {'Shapiro Index':<15} {'Classification':<15}")
    print("-" * 50)

    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            _, p_value = shapiro(df[col])

            if p_value < 0.05:
                classification = "Not Normal"
            else:
                classification = "Normal"

            print(f"{col:<20} {p_value:.4f} {classification:<15}")
        else:
            print(f"{col:<20} N/A {'Not applicable':<15}")
    print('Recheck for non normal data before ttest/ ANOVA after outlier')

"""## Data Transformation Function
This codeblock has all outlier treatment and imputation function
"""

def handle_missing_values(data, method="mean"):
    """Handles missing values using specified method ('mean', 'median', or 'drop')."""
    if method == "mean":
        return data.fillna(data.mean())
    elif method == "median":
        return data.fillna(data.median())
    elif method == "drop":
        return data.dropna()
    else:
        raise ValueError("Invalid method. Supported methods: 'mean', 'median', 'drop'")
# Outlier treatment
"""
    Detects outliers in a numerical column using IQR and removes or winsorizes them.

    Args: data frame

    Returns:        The cleaned df with outliers removed or winsorized.
        """


def treat_outliers_clip(df,exclude_list):

    # Identify numerical columns for analysis
    numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.difference(exclude_list)


    # Create a function for outlier detection and treatment
    def detect_outliers(column):


        # Calculate IQR and quantiles
        q1 = column.quantile(0.25)
        q3 = column.quantile(0.75)
        iqr = q3 - q1

        # Identify potential outliers (values beyond 1.5 IQR from quartiles)
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = column[(column < lower_bound) | (column > upper_bound)]
        #Clip data to upper and lower bounds
        cleaned_column = column.clip(lower_bound, upper_bound,axis=0)

        return cleaned_column
    clean_df=df.copy()
# Iterate through numerical columns and visualize distributions
    for column in numerical_columns:
        print(f"\nAnalyzing column: {column}")
        clean_df[column] = detect_outliers(df[column])
    return clean_df

# Winsorize function
from scipy.stats import mstats

def winsorize_numeric_columns(df, exclude_col=None, alpha=0.05):
    """
    Winsorizes numeric columns in the DataFrame (excluding specified columns).

    Args:
        df (pd.DataFrame): Input DataFrame.
        exclude_col (list or None): List of column names to exclude (optional).
        alpha (float): Winsorization parameter (default is 0.05).

    Returns:
        pd.DataFrame: Cleaned DataFrame with winsorized numeric columns.
    """
    if exclude_col is None:
        exclude_col = []  # Initialize as an empty list if not provided

    # Select numeric columns excluding specified ones
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.difference(exclude_col)

    # Apply winsorization to selected numeric columns
    for col in numeric_cols:
        df[col] = mstats.winsorize(df[col], limits=[alpha, alpha])

    return df

"""## Data Visualisation Function

"""

# Box Plot and histogrram Side by Side takes two df as parameters and a exclude list
def compare_distributions(data, df_cleaned,exclude_list=None):

    numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns.difference(exclude_list)

    for col in numerical_columns:
        print(f"\nAnalyzing column: {col}")

        # Set Plot size
        plt.figure(figsize=(15, 8))

        # Original DataFrame Boxplot
        plt.subplot(1, 2, 1)
        sns.boxplot(data=data[col])
        plt.title(f'Original DataFrame - {col}')

        # Cleaned DataFrame Boxplot
        plt.subplot(1, 2, 2)
        sns.boxplot(data=df_cleaned[col])
        plt.title(f'Cleaned DataFrame - {col}')

        plt.tight_layout()
        plt.show()

        # Separate Figure for Histograms
        plt.figure(figsize=(15, 8))

        # Original DataFrame Histogram
        plt.subplot(1, 2, 1)
        sns.histplot(data=data[col], kde=True)
        plt.title(f'Original DataFrame - {col}')

        # Cleaned DataFrame Histogram
        plt.subplot(1, 2, 2)
        sns.histplot(data=df_cleaned[col], kde=True)
        plt.title(f'Cleaned DataFrame - {col}')

        plt.tight_layout()
        plt.show()

# Plot correlation matrix
def plot_corr_matrix(df,predictors):

    corr_matrix = df[predictors].corr()

    # Heatmap visualization

    # Increase size
    plt.figure(figsize=(12, 8))  # Adjust width and height as desired

    # Select a colormap:
    cmap = sns.color_palette("coolwarm", as_cmap=True)  # Try other palettes!

    # Create the heatmap
    sns.heatmap(corr_matrix, annot=True, cmap=cmap)
    plt.title('Correlation Heatmap Enhanced')
    plt.show()
    # Convert the correlation matrix to a DataFrame (if it isn't already)
    if not isinstance(corr_matrix, pd.DataFrame):
        corr_matrix = pd.DataFrame(corr_matrix)

    # Export to Excel
    corr_matrix.to_excel('/content/drive/My Drive/Dataset/' +'correlation_matrix.xlsx', index=True)
    # 'index=True' will include the row and column labels

"""## Machine Learning function"""

# @title Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_model_regress(df, target_variable, feature_cols, ordinal_features, onehot_features, exclude_col,models):
    X = df[feature_cols]
    y = df[target_variable]
    # Track metrics
    mse_scores = []
    r2_scores = []
    # Train-test split using stratified sampling
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Preprocessing Pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('oe_type', OrdinalEncoder(categories='auto'), ordinal_features),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False), onehot_features),
            ('scaler', StandardScaler(), [col for col in feature_cols if col not in ordinal_features + onehot_features+ exclude_col])
        ],
        remainder='passthrough'
    )

    # Build the final Pipeline
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', models)
    ])

    # Fit the pipeline
    pipe.fit(X_train, y_train)

    # Make predictions
    y_pred = pipe.predict(X_test)

    # Calculate mean squared error
    # Calculate metrics
    mse_scores.append(mean_squared_error(y_test, y_pred))
    r2_scores.append(r2_score(y_test, y_pred))
    # ... (Rest of the function for feature importances and plotting)

    return mse_scores, r2_scores # Return MSE and predictions
    # Bar chart of feature of importace if tree based algo choseen
    # Feature importances (applicable to tree-based models)
    if hasattr(models, 'feature_importances_'):
      feature_importances = pd.Series(models.feature_importances_, index=feature_cols)
      feature_importances.sort_values(ascending=False, inplace=True)

      # Plot feature importances
      plt.figure(figsize=(10, 6))
      sns.barplot(x=feature_importances.index, y=feature_importances.values)
      plt.title(f"Feature Importances: {models.__class__.__name__}")
      plt.xticks(rotation=45)
      plt.tight_layout()
      plt.show()

# @title Stratified fold Split with classifier algo- Under progress
# Cant import stratified Kfold in colab version clash. Download file from github
''''
# Using K stratified fold
import sklearn
print(sklearn.__version__)
!pip install scikit-learn-contrib-stratified-kfold==0.2

from sklearn_contrib.stratified_kfold import KStratifiedFold


def evaluate_model_classify(df, target_variable, feature_cols, ordinal_features, onehot_features, model):
    X = df[feature_cols]
    y = df[target_variable]

    # KStratifiedFold for cross-validation
    cv = KStratifiedFold(n_splits=5, shuffle=True, random_state=42)

    # Track metrics
    mse_scores = []
    r2_scores = []
    accuracy_scores = []

    for train_index, test_index in cv.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Preprocessing Pipeline
        preprocessor = ColumnTransformer(
        transformers=[
            ('oe_type', OrdinalEncoder(categories='auto'), ordinal_features),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False), onehot_features),
            ('scaler', StandardScaler(), [col for col in feature_cols if col not in ordinal_features + onehot_features])
        ],
        remainder='passthrough'
        )
    # Build the final Pipeline
        pipe = Pipeline(steps=[
          ('preprocessor', preprocessor),
          ('model', model)
        ])

        # Fit the pipeline
        pipe.fit(X_train, y_train)

        # Make predictions
        y_pred = pipe.predict(X_test)

        # Calculate metrics
        mse_scores.append(mean_squared_error(y_test, y_pred))
        r2_scores.append(r2_score(y_test, y_pred))

    return mse_scores, r2_scores, y_pred
    '''

"""

"""


###Frequently used transformation

In [ ]:
# Function returns numeric column list minus the excluded column. Very quick from reselecting data fields

def get_numeric_columns(df, exclude_columns=None):
    """
    Returns a list of numeric column names from the DataFrame, excluding specified columns.

    Args:
        df (pd.DataFrame): Input DataFrame.
        exclude_columns (list or None): List of column names to exclude (optional).

    Returns:
        list: List of numeric column names.
    """
    if exclude_columns is None:
        exclude_columns = []  # Initialize as an empty list if not provided

    numeric_columns = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
    filtered_numeric_columns = [col for col in numeric_columns if col not in exclude_columns]

    return filtered_numeric_columns

# Function converts columns to snake case
import re

def convert_columns_to_snake_case(df):
    """Converts all column names in a DataFrame to snake_case.

    Args:
         DataFrame.

    Returns:
        The DataFrame with modified column names.
    """
    # Modify depending on what you want to delete
    def to_snake_case(name):
        name = re.sub(r'[^\w_]', '', name)  # Remove special characters except alphanumeric, underscore
        name = name.replace(' ', '_')  # Replace spaces with underscore
        return name.lower()  # Convert to lowercase

    df.rename(columns=to_snake_case, inplace=True)
    return df